In [1]:
# Check instance GPUs
!nvidia-smi

Thu Aug 24 02:11:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:1B.0 Off |                    0 |
| N/A   37C    P0    50W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:00:1C.0 Off |                    0 |
| N/A   

In [2]:
# %%capture
# Install required libraries
%pip install -q huggingface_hub
%pip install -q -U trl transformers accelerate bitsandbytes xformers
%pip install -q -U datasets einops wandb evaluate sentencepiece

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%%capture
# Install git lfs
%conda install --yes -c conda-forge git-lfs

%git lfs install --yes

In [4]:
# Import Libraries

from datasets import load_dataset
import torch
from transformers import Trainer, TrainingArguments
import wandb
from huggingface_hub import notebook_login
import evaluate
import numpy as np
from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import AutoTokenizer, LlamaTokenizer
from transformers import DataCollatorWithPadding
import os

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# Log in to WandB
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ingeniousartist. Use `wandb login --relogin` to force relogin


True

In [6]:
# Create WandB project
%env WANDB_PROJECT=sentiment_finance

env: WANDB_PROJECT=sentiment_finance


In [7]:
# Log in to your Hugging Face account
# Get your API token here https://huggingface.co/settings/token
notebook_login()

In [9]:
# Load dataset
from datasets import load_dataset
dataset = load_dataset("financial_phrasebank","sentences_50agree", split='train')

In [10]:
# Create train and test datasets
train_dataset = dataset.select([i for i in list(range(0,3873))])
test_dataset = dataset.select([i for i in list(range(3873,4841))])

print(train_dataset[0])
print(test_dataset[0])

{'sentence': 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .', 'label': 1}
{'sentence': 'The ongoing project where Tekla Structures is being used is the Vashi Exhibition Centre being developed by Insteel Engineers Pvt Ltd-IIVRCL Infrastructures & Projects Ltd & CIDCO .', 'label': 1}


In [11]:
# Load Metric

accuracy = evaluate.load("accuracy")

In [12]:
# Create compute metrics class

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    
    # Convert predictions and labels into int32 from int64
    predictions = predictions.astype(np.int32)
    labels = labels.astype(np.int32)
    
    # Compute metric
    accuracy_res = accuracy.compute(predictions=predictions, references=labels)
    return accuracy_res

In [13]:
# Define StableLM-base as our base model:
base_model_name = "openlm-research/open_llama_3b_v2"

# Add names of multiclass labels
label2id = {
    "Negative": 0,
    "Neutral": 1,
    "Positive": 2
  }
id2label = {
    "0": "Negative",
    "1": "Neutral",
    "2": "Positive"
  }

# Add labels to model config
config = AutoConfig.from_pretrained(base_model_name, label2id=label2id, id2label=id2label)

In [14]:
# Set device map according to instance
device_map = "auto"
# device_map = {'': 0}

In [15]:
# Create model object

model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True,
    config=config,
    # max_memory=f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB' # For avoiding CUDA out of memory (single GPU)
)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:460: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at openlm-research/open_llama_3b_v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Add more settings to model config

model.config.use_cache = False
model.config.pretraining_tp = 1 

In [17]:
# Create a function to get parameters of the model

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(model)

trainable params: 3324083200 || all params: 3324083200 || trainable%: 100.0


In [18]:
# Initialize tokenizer object from pre-trained model
# tokenizer = AutoTokenizer.from_pretrained("openlm-research/open_llama_3b_v2")
tokenizer = LlamaTokenizer.from_pretrained("openlm-research/open_llama_3b_v2")

# convert all the model weights to half precision
#model.half().cuda()

# Add padding and test tokenizer
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
print(tokenizer.encode("Hello this is a test"))

# Add padding to model config
model.config.pad_token_id = model.config.eos_token_id

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


[1, 8479, 420, 325, 260, 1248]


In [19]:
# Create tokenized train and test datasets

train_dataset = train_dataset.map(lambda samples: tokenizer(samples["sentence"]), batched=True)
test_dataset = test_dataset.map(lambda samples: tokenizer(samples["sentence"]), batched=True)

In [20]:
# Create data collator with padding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [21]:
# Set repository name and output folder

repo_name = "openllama-3b-finance"
output_dir = repo_name

# Set training args
training_args = TrainingArguments(
    output_dir=output_dir,
    
    # Low batch size to avoid OOM
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=2e-4,
    logging_steps=20, # Logs every 20 steps and measures everything
    weight_decay=0.001,
    eval_accumulation_steps=10, # Low accumulation to avoid OOM
    num_train_epochs=1,
    lr_scheduler_type = "cosine", # Schedules linear/cosine
    evaluation_strategy="steps", # Evaluates every 20 steps
    save_strategy="no",
    #save_steps = 500,
    seed = 42,
    save_safetensors = True,
    push_to_hub=True,
    gradient_checkpointing = True,
    auto_find_batch_size = True, # Good for avoiding OOM
    optim="paged_adamw_8bit", # Optimizer
    
    # other args and kwargs here
    report_to="wandb",  # enable logging to W&B
    run_name="openllama-3b (cosine)",  # name of the W&B run (optional)
)

# Set Sequence length
max_seq_length = 512

# Initialize trainer object with args and dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [22]:
# Train the model and evaluate according to strategy

trainer.train()

Step,Training Loss,Validation Loss,Accuracy
20,21.965500,8.166256,0.081612
40,2.231000,6.300709,0.414256
60,2.745200,4.089234,0.414256
80,2.456100,5.031425,0.414256
100,2.337000,5.617582,0.414256
120,3.222600,4.496350,0.414256
140,2.563300,6.179950,0.414256
160,2.476400,4.705860,0.414256
180,2.450000,5.060175,0.414256
200,1.423200,5.341760,0.414256


TrainOutput(global_step=3873, training_loss=2.663619207160097, metrics={'train_runtime': 11048.6646, 'train_samples_per_second': 0.351, 'train_steps_per_second': 0.351, 'total_flos': 2509046876160000.0, 'train_loss': 2.663619207160097, 'epoch': 1.0})

In [23]:
# Empties GPU cache to avoid OOM

torch.cuda.empty_cache()

In [24]:
# Finishes run logging on WandB

wandb.finish()

eval/accuracy,████▁███████▁███████████████████████████
eval/loss,▁▁▁▂█▇▆▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▆▂▂▂▂▂▇▇▇▇▃▂▂▂▃▂▂▃▃▂▃▂▂▂▁▄▃▂▄▂▇███▃▃▃▃▂▃
eval/samples_per_second,▃▇▇▇▇▇▂▂▂▂▆▆▇▇▆▇▇▆▆▆▆▆▇▇█▄▆▇▅▇▂▁▁▁▆▆▆▆▇▆
eval/steps_per_second,▃▇▇▇▇▇▂▂▂▂▆▆▇▇▆▇▇▆▆▆▆▆▇▇█▄▆▇▅▇▂▁▁▁▆▆▆▆▇▆
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███████▇▇▇▇▇▆▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train/loss,▂▂▁▁▂█▄▁▁▂▂▂▁▂▁▂▂▂▂▂▂▂▂▁▂▂▂▂▁▁▁▂▁▁▂▂▁▁▁▁
train/total_flos,▁
train/train_loss,▁


In [25]:
# Saves final checkpoint, model and tokenizer

output_dir = os.path.join(output_dir, "final_checkpoint")
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('openllama-3b-finance/final_checkpoint/tokenizer_config.json',
 'openllama-3b-finance/final_checkpoint/special_tokens_map.json',
 'openllama-3b-finance/final_checkpoint/tokenizer.model',
 'openllama-3b-finance/final_checkpoint/added_tokens.json')

In [26]:
# Upload the model to the Hub
# model.push_to_hub("openllama-3b-finance")
# tokenizer.push_to_hub("openllama-3b-finance")
trainer.push_to_hub("openllama-3b-finance")

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.16k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.31G [00:00<?, ?B/s]

'https://huggingface.co/IngeniousArtist/openllama-3b-finance/tree/main/'